In [13]:
!pip install transformers[torch]
!pip install datasets

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [14]:
from transformers import DistilBertForSequenceClassification, DistilBertTokenizerFast

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
from datasets import load_dataset

raw_datasets = load_dataset("imdb")

Found cached dataset imdb (C:/Users/FARMDAR/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0)


  0%|          | 0/3 [00:00<?, ?it/s]

In [16]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)

Loading cached processed dataset at C:\Users\FARMDAR\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-b9c08d3984c3008f.arrow
Loading cached processed dataset at C:\Users\FARMDAR\.cache\huggingface\datasets\imdb\plain_text\1.0.0\d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0\cache-9f3640403b2ace0e.arrow


Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [24]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    "test_trainer",  # Specify your desired output directory
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size= 20,
    num_train_epochs=3,
    weight_decay=0.01,
    max_steps=10,  # Set the maximum number of training steps to 200
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss
0,No log,0.696387


TrainOutput(global_step=10, training_loss=0.7106836318969727, metrics={'train_runtime': 4543.2921, 'train_samples_per_second': 0.009, 'train_steps_per_second': 0.002, 'total_flos': 5298695946240.0, 'train_loss': 0.7106836318969727, 'epoch': 0.0})

In [ ]:
trainer.evaluate()

In [26]:
import torch

text = "This movie was amazing!"
inputs = tokenizer(text, return_tensors="pt")
logits = model(**inputs).logits
predicted_label = torch.argmax(logits, dim=1)

print(predicted_label)

tensor([0])
